### Решение вступительного задания хакатона М.Видео с использованием сверточной нейронной сети с представлениями слов от Facebook

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import string
import gensim
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Input
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
raw_data = pd.read_csv("X_train.csv")

In [3]:
# Перемешаем наши данные.
raw_data = raw_data.sample(frac=1)

In [4]:
raw_data[:3]

,sku,categoryLevel1Id,categoryLevel2Id,brandId,property,userName,reting,date,comment,commentNegative,commentPositive
9313,20004410,413,4130301,10,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",def22edab63005cefd72998769aab8e5,4.7,2017-03-01,"Легкая, почти безшумная, вибрация не чувствует...",Нет,Купили сыну в подарок на 23 февраля. Стрижемся...
3993,20025733,403,4030101,57,"[{3: '3ef815416f775098fe977004015c6193'}, {1: ...",37339a5698d5e3ea0b50dd53c319f8b4,4.0,2013-03-29,Купили машинку в октябре и с радостью пользуем...,NaN,NaN
9992,20004251,413,4130201,759,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",6738f9acd4740d945178c649d6981734,5.0,2013-04-30,"Подарили на восьмое марта, пользуюсь! Все рабо...",NaN,NaN


In [5]:
raw_data = raw_data.rename(columns={'reting':'rating'})

In [6]:
raw_data.isnull().any()

sku                 False
categoryLevel1Id    False
categoryLevel2Id    False
brandId             False
property            False
userName            False
rating              False
date                False
comment             False
commentNegative      True
commentPositive      True
dtype: bool

In [7]:
raw_data.shape

(15587, 11)

In [8]:
raw_data = raw_data.fillna(value=' ')

In [9]:
raw_data.sku.unique().shape

(2698,)

In [10]:
raw_data.categoryLevel1Id.unique().shape

(48,)

In [11]:
raw_data.categoryLevel2Id.unique().shape

(164,)

In [12]:
raw_data.brandId.unique().shape

(193,)

In [13]:
raw_data.rating.unique()

array([ 4.7,  4. ,  5. ,  2. ,  3. ,  1. ,  2.7,  4.3,  3.3,  3.7,  2.3,
        1.3,  1.7])

** После нескольких тестов мы поняли что категориальные признаки почти не коррелируют с ответом, поэтому далее мы их учитывать не будем.**

** Заметили что рейтинги бывают и дробными, округлим их до целых и будем решать задачу классификации.**

In [14]:
raw_data.rating = raw_data.rating.astype(int)

In [15]:
raw_data.rating.unique()

array([4, 5, 2, 3, 1])

In [16]:
usefull_columns = ['sku', 'categoryLevel1Id', 
                   'categoryLevel2Id', 'brandId', 
                   'comment', 'commentNegative', 'commentPositive']

In [17]:
data = raw_data[usefull_columns]
target = raw_data['rating']

In [18]:
print('1: ', np.sum(target==1), '2: ', np.sum(target==2), '3: ', np.sum(target==3), '4: ', 
      np.sum(target==4), '5: ', np.sum(target==5))

1:  1475 2:  883 3:  1277 4:  2741 5:  9211


** Сделаем target от 0 до 4, для многоклассовой классификации.**

In [19]:
target = target - 1

In [20]:
data_len = data.shape[0]
class_weight = {0 : np.sum(target==0) / data_len,
    1: np.sum(target==1) / data_len,
    2: np.sum(target==2) / data_len,
    3: np.sum(target==3) / data_len,
    4: np.sum(target==4) / data_len,
    }

** Итак, мы получили данные без nan, разделенные на тренировочную и тестовую выборки.**

In [21]:
bag_data = data[data.columns[-3:]]

In [22]:
# Сделаем одну колонку, содержащую всю текстовую информацию.
bag_data['words'] = (bag_data.comment.astype(str) 
                     + ' ' + bag_data.commentNegative.astype(str) 
                     + ' ' + bag_data.commentPositive.astype(str))

/home/vprov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


** Оставим одну колонку, где будет содержаться вся текстовая информация из отзыва. **

In [23]:
bag_data = bag_data['words']

** Удалим все знаки препинания. **

In [25]:
exclude = set(string.punctuation)

In [26]:
bag_data = bag_data.values

In [27]:
for i,_ in enumerate(bag_data):
    bag_data[i] = ''.join(ch for ch in bag_data[i] if ch not in exclude)

** Загрузим натренированные заранее представления слов от Facebook. 
Их можно найти по этой ссылке: https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md**

In [31]:
%%time
en_model = KeyedVectors.load_word2vec_format('data/wiki.ru.bin')

CPU times: user 12min 27s, sys: 5.82 s, total: 12min 32s
Wall time: 12min 38s


In [36]:
en_model.word_vec("как")

array([  3.84939983e-02,   2.77289987e-01,   8.55579972e-02,
        -6.22979999e-02,   1.06619999e-01,  -3.61369997e-02,
         8.60600024e-02,   5.43989986e-03,   7.60649983e-03,
         7.15039996e-03,  -5.65249994e-02,   2.36599997e-01,
         3.27520013e-01,   2.85109997e-01,  -2.72850007e-01,
        -1.72869995e-01,  -2.58990005e-02,  -3.16179991e-01,
         1.58270001e-01,   2.57319987e-01,   2.44389996e-01,
         8.20790008e-02,  -1.01499997e-01,   5.65720014e-02,
         9.13420022e-02,   1.11769997e-01,  -2.24329997e-02,
        -5.49070016e-02,   9.73820016e-02,  -3.89619991e-02,
        -1.27869993e-01,  -2.31630006e-03,   1.03009999e-01,
        -1.07210003e-01,  -6.75449986e-03,  -1.22249998e-01,
         8.51989985e-02,  -1.02899998e-01,   3.64840001e-01,
        -6.63900003e-02,   2.37029999e-01,  -1.96730003e-01,
        -3.88859987e-01,  -1.68809995e-01,   1.48949996e-01,
        -1.44060001e-01,  -2.35819995e-01,  -5.32040000e-02,
        -1.67740002e-01,

In [38]:
tokenizer = Tokenizer()

In [39]:
tokenizer.fit_on_texts(bag_data)

In [40]:
sequences = tokenizer.texts_to_sequences(bag_data)

In [41]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 71732 unique tokens.


In [42]:
# Сделали последовательность (num_samples, num_timestamps)
MAX_SEQUENCE_LENGTH = 50
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [43]:
data.shape

(15587, 50)

In [44]:
labels = to_categorical(target)

In [46]:
# Мы используем заранее натренированные векторы размера 300.
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))

In [64]:
embeddings_index = {}

Не все слова есть в этой модели.

In [52]:
for word, i in word_index.items():
    if word in en_model.vocab:
        embedding_vector = en_model.word_vec("как")
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [54]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

In [55]:
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.core import Dense
from keras.layers.core import Flatten
from keras.models import Model
from keras import optimizers

** Решим задачу с помощью сверточной нейронной сети. Эти модели хорошо зарекомендовали себя на обработке текстов.**

In [56]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            #trainable=False
                           )

In [57]:
embedded_sequences = embedding_layer(sequence_input)

In [58]:
l_cov1= Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(3)(l_cov1)
l_cov2 = Conv1D(128, 3, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(3)(l_cov2)
l_cov3 = Conv1D(128, 3, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(2)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(5, activation='softmax')(l_dense)

In [59]:
model = Model(sequence_input, preds)

In [60]:
#Adam = optimizers.Adam(lr=0.05)
#prop = optimizers.rmsprop(lr=0.05)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              #optimizer=Adam,
              metrics=['acc'])

In [61]:
embedding_matrix.shape

(71733, 300)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=5)

In [63]:
X_train.shape

(10910, 50)

In [64]:
y_train.shape

(10910, 5)

In [65]:
%%time
model.fit(X_train, y_train,
          epochs=10, 
          class_weight=class_weight,
          batch_size=128)

Epoch 1/10
10910/10910 [==============================] - 28s - loss: 0.2231 - acc: 0.5821    
Epoch 2/10
10910/10910 [==============================] - 15s - loss: 0.2051 - acc: 0.5884    
Epoch 3/10
10910/10910 [==============================] - 15s - loss: 0.1929 - acc: 0.5885    
Epoch 4/10
10910/10910 [==============================] - 15s - loss: 0.1729 - acc: 0.5933    
Epoch 5/10
10910/10910 [==============================] - 15s - loss: 0.1547 - acc: 0.6281    
Epoch 6/10
10910/10910 [==============================] - 15s - loss: 0.1376 - acc: 0.6520    
Epoch 7/10
10910/10910 [==============================] - 15s - loss: 0.1222 - acc: 0.6831    
Epoch 8/10
10910/10910 [==============================] - 15s - loss: 0.1076 - acc: 0.7079    
Epoch 9/10
10910/10910 [==============================] - 15s - loss: 0.0953 - acc: 0.7300    
Epoch 10/10
10910/10910 [==============================] - 15s - loss: 0.0830 - acc: 0.7515    
CPU times: user 2min, sys: 36 s, total: 2min 36s


In [66]:
pred = model.predict(X_test)

** Предсказание модели для каждого комментария это вектор размера 5(количество классов), где в каждой из ячеек написана вероятность принадлежности этому классу.**

In [67]:
prediction = []
for raw in pred:
    prediction.append(np.argmax(raw))

In [68]:
prediction = np.array(prediction)

In [69]:
prediction[:5]

array([4, 4, 3, 0, 4])

In [70]:
true = []
for raw in y_test:
    true.append(np.argmax(raw))

In [71]:
true = np.array(true)

In [72]:
true[:5]

array([1, 4, 4, 0, 3])

In [73]:
accuracy_score(y_pred=prediction, y_true=true)

0.59119093435963221

Но сложно что-то сказать по этой метрике, поэтому сделаем метрику поподробнее.

In [74]:
def get_metrics(prediction, y_true):
    for i in range(5):
        true_positive = 1
        false_positive = 1
        true_negative = 1
        false_negative = 1
        for ind, ans in enumerate(prediction):
            if ans == i and y_true[ind]==i:
                true_positive += 1
            elif ans == i and y_true[ind] !=i:
                false_positive += 1
            elif ans != i and y_true[ind]==i:
                false_negative += 1
            else:
                true_negative += 1
        print("Class number: ", i, "true_positive: ", true_positive, "false positive: ", false_positive, '\n',
             "false negative: ", false_negative, "true negative: ", true_negative)
        print("Accuracy: ", (true_positive + true_negative) / (true_positive + false_positive + true_negative +
                                                              false_negative))
        print("Recall: ", true_positive / (true_positive + false_negative))
        print("Precision: ", true_positive / (true_positive + false_positive))

In [75]:
get_metrics(prediction, true)

Class number:  0 true_positive:  110 false positive:  191 
 false negative:  344 true negative:  4036
Accuracy:  0.8857081820123905
Recall:  0.2422907488986784
Precision:  0.3654485049833887
Class number:  1 true_positive:  1 false positive:  1 
 false negative:  254 true negative:  4425
Accuracy:  0.945524460585345
Recall:  0.00392156862745098
Precision:  0.5
Class number:  2 true_positive:  4 false positive:  8 
 false negative:  391 true negative:  4278
Accuracy:  0.9147618030335398
Recall:  0.010126582278481013
Precision:  0.3333333333333333
Class number:  3 true_positive:  191 false positive:  668 
 false negative:  598 true negative:  3224
Accuracy:  0.7295449690237129
Recall:  0.2420785804816223
Precision:  0.22235157159487776
Class number:  4 true_positive:  2464 false positive:  1049 
 false negative:  330 true negative:  838
Accuracy:  0.7054048280281991
Recall:  0.8818897637795275
Precision:  0.7013948192428124


** Вывод: Нам удалось построить сеть, которая довольно хорошо классифицирует отзывы.
Лучше всего наша сеть предсказывает последний класс(оценка 5), этого класса на много больше чем остальных и в тестовых и в тренировочных данных. Она плохо определяет классы с оценками 3 и 4, но как видно очень плохие отзывы она видит и классифицирует с оценкой 1.**